## News Article Similarity Modelling
- Cross encoding 
- Translated data 
- Using Title 

In [1]:
from tqdm.auto import tqdm
import torch 
import random
from torch import nn
from transformers import RobertaTokenizer, PreTrainedTokenizer, DistilBertTokenizer, DistilBertModel, RobertaModel
from sentence_transformers import SentenceTransformer, SentencesDataset, InputExample, losses
from datasets import Dataset
import pandas as pd
from transformers.optimization import get_linear_schedule_with_warmup
import matplotlib.pyplot as plt 
import numpy as np
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModel
from torch.nn import CosineEmbeddingLoss
import transformers
import seaborn as sns
#Build up to SBERT model 

In [2]:
deviceNum = 1
device = torch.device("cuda:" + str(deviceNum) if torch.cuda.is_available() else "cpu")

In [3]:
GRAD_ACC = 6
EPOCHS = 3
FOLDS = 5
SEED = 85
RDROP_WEIGHT = .1
FORWARD_WEIGHT = (1 - RDROP_WEIGHT) / 2
BATCH_SIZE = 5

In [4]:
def check_mem():
    torch.cuda.empty_cache()
    a = torch.cuda.memory_allocated(deviceNum)/1024/1024/1024
    r = torch.cuda.memory_reserved(deviceNum)/1024/1024/1024
    print("torch.cuda.memory_allocated: %fGB"%a)
    print("torch.cuda.memory_reserved: %fGB"%r)
    print("torch.cuda.memory_free: %fGB"%(r-a))
    print("torch.cuda.max_memory_reserved: %fGB"%(torch.cuda.max_memory_reserved(deviceNum)/1024/1024/1024))
check_mem()

torch.cuda.memory_allocated: 0.000000GB
torch.cuda.memory_reserved: 0.000000GB
torch.cuda.memory_free: 0.000000GB
torch.cuda.max_memory_reserved: 0.000000GB


In [5]:
#set seeds 
torch.manual_seed(85)
random.seed(85)
torch.backends.cudnn.deterministic = False

In [6]:
#test dropout 
ones = torch.ones(200)
dropout = nn.Dropout(.5)
GELU = nn.GELU()
print(dropout(ones))
print(dropout(ones))


tensor([2., 2., 2., 2., 2., 2., 0., 2., 2., 0., 0., 2., 0., 2., 0., 2., 0., 2.,
        0., 0., 2., 0., 0., 0., 0., 2., 0., 0., 0., 2., 2., 0., 2., 0., 0., 2.,
        2., 0., 2., 2., 0., 2., 0., 0., 2., 0., 0., 2., 0., 2., 2., 2., 2., 0.,
        2., 0., 2., 0., 2., 2., 2., 0., 0., 0., 2., 0., 2., 2., 0., 2., 0., 0.,
        2., 2., 2., 2., 0., 0., 2., 2., 0., 2., 0., 2., 0., 2., 0., 2., 0., 0.,
        2., 2., 2., 0., 0., 2., 2., 0., 2., 2., 0., 2., 2., 0., 2., 2., 0., 2.,
        0., 2., 0., 0., 0., 2., 0., 2., 0., 0., 2., 2., 0., 0., 2., 0., 0., 2.,
        2., 0., 0., 2., 2., 0., 2., 0., 0., 0., 0., 0., 2., 0., 2., 2., 0., 2.,
        0., 0., 0., 0., 0., 2., 0., 2., 0., 0., 2., 0., 0., 0., 2., 0., 0., 0.,
        0., 0., 2., 0., 0., 0., 2., 2., 2., 0., 0., 0., 2., 0., 2., 0., 2., 0.,
        0., 2., 0., 0., 0., 2., 0., 0., 2., 2., 2., 0., 0., 2., 0., 0., 2., 2.,
        0., 0.])
tensor([0., 2., 0., 2., 0., 0., 2., 2., 2., 2., 2., 0., 0., 0., 2., 0., 2., 2.,
        2., 2., 2., 0.,

In [7]:
df = pd.read_csv("/home/blitt/projects/localNews/data/processed/translated_200_56.tsv", sep="\t")
#df = df.loc[(df["url1_lang"] == "en") & (df["url2_lang"] == "en")]

#NOTE: may need to add categories back in at some point 
groundTruths = ["Geography", "Entities","Time", "Narrative", "Overall"]
features = ['text1Merged', 'text2Merged','url1_lang', 'url2_lang']
toSelect = groundTruths + features 

#get only the columns we need 
#TODO: do we need "pair_id"? 
leanDf = df[toSelect].dropna()

#rescale data from (0, 4): (0, 1)
for colName in groundTruths: 
    leanDf[colName] = 1 - ((leanDf[colName] - 1) / 3)

#reset index so it is contiguous set of numbers 
leanDf = leanDf.reset_index(drop=True)


In [8]:
class Model(nn.Module): 
    def __init__(self):
        super(Model,self).__init__()
        self.model = RobertaModel.from_pretrained('roberta-base')
        self.l1 = nn.Linear(768, 256).to(device)
        self.l2 = nn.Linear(256, 5)
        self.GELU = nn.GELU()
        self.loss_func = torch.nn.MSELoss(reduction="mean")
        
    def mean_pooling(self, token_embeddings, attention_mask): 
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

    def forward(self, input_ids, attention_mask): 
        
        #encode sentence and get mean pooled sentence representation 
        encoding = self.model(input_ids, attention_mask=attention_mask)[0]  #all token embeddings
        meanPooled = self.mean_pooling(encoding, attention_mask)
       
        pred1 = self.l2(self.GELU(self.l1(meanPooled)))
        
        encoding = self.model(input_ids, attention_mask=attention_mask)[0]  #all token embeddings
        meanPooled = self.mean_pooling(encoding, attention_mask)
       
        pred2 = self.l2(self.GELU(self.l1(meanPooled)))

        return [pred1, pred2]


In [9]:
"""
Get the loss across multiple different objectives. 
Since overall is most important it gets more weight. 
"""
def getWeightedLoss(predTens, gtTens):
    loss_func = torch.nn.MSELoss(reduction="mean")
    #try getting rid of Tone and Style 
    LOSS_WEIGHTS = [.1, .1, .1, .1, .6]
    loss = 0.0
    for i in range(len(LOSS_WEIGHTS)): 
        
        #get ground truth value associated with this column name 
        currGT = gtTens[:, :, i]
        
        #TODO: figure out how to index properly here 
        pred = predTens[:, :, i]
        
        #get loss 
        loss += (loss_func(pred, currGT) * LOSS_WEIGHTS[i])
    return loss 

In [10]:
def validation(model, validLoader, loss_func): 
    model.eval()
    lossList = []
    predList = []
    GT = []
    
    i = True 
    for index, batch in enumerate(validLoader): 

        # prepare batches and more all to the active device
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        #label = batch['ground_truth'].to(device).unsqueeze(1)

        #send batch info through model 
        pred1, pred2 = model(input_ids, attention_mask)
        pred1, pred2 = pred1.unsqueeze(0), pred2.unsqueeze(0)
        
        #get ground truths 
        gts = torch.stack([batch[colName] for colName in groundTruths], 0).to(device).T.unsqueeze(0)
        
        #get weighted loss relating to label prediction 
        loss1 = getWeightedLoss(gts, pred1) * FORWARD_WEIGHT 
        loss2 = getWeightedLoss(gts, pred2) * FORWARD_WEIGHT
        loss_r = getWeightedLoss(pred1, pred2) * RDROP_WEIGHT
        loss = (loss1 + loss2 + loss_r) / GRAD_ACC
        
        #get output metrics 
        #does it matter that we only evaluate on pred1? 
        lossList.append(loss.detach().cpu().item())
        predList.append([float(item) for item in list(pred1.detach().cpu().squeeze())])
        GT.append([float(item) for item in list(gts.detach().cpu().squeeze())])
        
        del pred1, pred2, gts, loss
    #print(vGT)
    return [lossList, predList, GT]
        

In [11]:
#set up relevant variables 
def train(trainDataset, validDataset): 
    torch.cuda.empty_cache()
    #get loaders 
    trainLoader = torch.utils.data.DataLoader(
        trainDataset, batch_size=BATCH_SIZE, shuffle=True
    )
    validLoader = torch.utils.data.DataLoader(
        validDataset, batch_size=1, shuffle=True
    )
    
    trainLen = len(trainDataset)

    #load the model 
    model = Model().to(device)

    #TODO: double check on if reduction="mean" is the right move here...
    #could cosine similarity also work..? I think that is between the two predicted vectors though.. 
    loss_func = torch.nn.MSELoss(reduction="mean")

    # we would initialize everything first
    optim = torch.optim.Adam(model.parameters(), lr=2e-5)

    #set up scheduler
    # and setup a warmup for the first ~10% steps
    total_steps = int((trainLen*EPOCHS) / BATCH_SIZE)
    warmup_steps = int(0.1 * total_steps)
    scheduler = get_linear_schedule_with_warmup(optim, num_warmup_steps=warmup_steps, num_training_steps=total_steps - warmup_steps)
    
    
    #now run training loop 
    lossList = []
    validMetrics = []
    subLossList = []
    
    # increase from 1 epoch if need be 
    for epoch in range(EPOCHS):
        torch.cuda.empty_cache()
        model.train()  # make sure model is in training mode

        # initialize the dataloader loop with tqdm (tqdm == progress bar)
        loop = tqdm(trainLoader, leave=True)

        validMetrics.append(validation(model, validLoader, loss_func))
        model.train()

        for i, batch in enumerate(loop): 
            # zero all gradients on each new step
            optim.zero_grad()

            # prepare batches and more all to the active device
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch["attention_mask"].to(device)
           
            #send batch info through model 
            pred1, pred2 = model(input_ids, attention_mask)
            pred1, pred2 = pred1.unsqueeze(0), pred2.unsqueeze(0)

            #get gts 
            gts = torch.stack([batch[colName] for colName in groundTruths], 0).T.to(device).unsqueeze(0)
        
            #get weighted loss relating to label prediction 
            loss1 = getWeightedLoss(gts, pred1) * FORWARD_WEIGHT 
            loss2 = getWeightedLoss(gts, pred2) * FORWARD_WEIGHT
            loss_r = getWeightedLoss(pred1, pred2) * RDROP_WEIGHT
            loss = (loss1 + loss2 + loss_r) / GRAD_ACC
            
            # using loss, calculate gradients and then optimize
            loss.backward()
            optim.step()

            #get mean loss over last 20 batches 
            if i % 20 == 0: 
                lossList.append(np.mean(subLossList))
                subLossList = []
                pass

            subLossList.append(float(loss.detach().item()))
        
            # update learning rate scheduler
            scheduler.step()

            # update the TDQM progress bar
            loop.set_description(f'Epoch {epoch}')
            loop.set_postfix(loss=loss.item())
            del loss 
        print("torch.cuda.memory_allocated: %fGB"%(torch.cuda.memory_allocated(0)/1024/1024/1024))
        print("torch.cuda.memory_reserved: %fGB"%(torch.cuda.memory_reserved(0)/1024/1024/1024))
        print("torch.cuda.max_memory_reserved: %fGB"%(torch.cuda.max_memory_reserved(0)/1024/1024/1024))

    validMetrics.append(validation(model, validLoader, loss_func))
    return validMetrics 
    del model
    del trainLoader
    del validLoader

In [12]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=FOLDS, shuffle=True)

In [13]:
metrics = []
transformers.logging.set_verbosity_error()
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

#we only want to sample validation data from the pairs that are both english 
enDf = leanDf[(leanDf["url1_lang"] == "en") & (leanDf["url2_lang"] == "en")]

print("Total df len: " +  str(len(leanDf)))
print("English df len: " +  str(len(enDf)))
#we create splits based on the position (not the actual index) of rows in enDf
#the idea is to get a split of the english dataset to set aside and then 
#grab everything else in the en + translated dataset to train on 
for i, (train_index, valid_index) in enumerate(kf.split(enDf)): 
    
    #grab the rows in enDf corresponding to the positions of our split 
    validDf = enDf.iloc[valid_index]
    
    #now get the actual indicies that have been selected
    #and subtract the indices in trainDf away from those 
    remainingIndices = list(set(leanDf.index) - set(validDf.index))
    trainDf = leanDf.loc[remainingIndices]
    print("###### " + str(i).upper() + " ######")
    print("Train df len: " + str(len(trainDf)))
    print("Valid df len: " + str(len(validDf)))
    
    #get data loaded in properly 
    trainDataset = Dataset.from_pandas(trainDf)
    validDataset = Dataset.from_pandas(validDf)
    
    #NOTE: here we use the merged text
    trainDataset = trainDataset.map(lambda x: tokenizer(x["text1Merged"], x["text2Merged"], max_length=512, padding="max_length", truncation=True))
    validDataset = validDataset.map(lambda x: tokenizer(x["text1Merged"], x["text2Merged"], max_length=512, padding="max_length", truncation=True))
    

    #only need the input information 
    trainDataset = trainDataset.remove_columns(["text1Merged", "text2Merged", "__index_level_0__"])
    validDataset = validDataset.remove_columns(["text1Merged", "text2Merged", "__index_level_0__"])
    
    # convert dataset features to PyTorch tensors
    formatColumns = groundTruths + ["input_ids", "attention_mask"]
    validDataset.set_format(type='torch', columns=formatColumns)
    trainDataset.set_format(type='torch', columns=formatColumns)

    validMetrics = train(trainDataset, validDataset)
    metrics.append(validMetrics)
    
    del trainDataset
    del validDataset
    !nvidia-smi

Total df len: 4806
English df len: 1738
###### 0 ######
Train df len: 4458
Valid df len: 348


  0%|          | 0/4458 [00:00<?, ?ex/s]

  0%|          | 0/348 [00:00<?, ?ex/s]

  0%|          | 0/892 [00:00<?, ?it/s]

/opt/anaconda/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/anaconda/lib/python3.8/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


...

torch.cuda.memory_allocated: 0.000000GB
torch.cuda.memory_reserved: 0.000000GB
torch.cuda.max_memory_reserved: 0.000000GB


  0%|          | 0/892 [00:00<?, ?it/s]

torch.cuda.memory_allocated: 0.000000GB
torch.cuda.memory_reserved: 0.000000GB
torch.cuda.max_memory_reserved: 0.000000GB


  0%|          | 0/892 [00:00<?, ?it/s]

torch.cuda.memory_allocated: 0.000000GB
torch.cuda.memory_reserved: 0.000000GB
torch.cuda.max_memory_reserved: 0.000000GB
Thu Jan 19 01:35:27 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 465.19.01    CUDA Version: 11.3     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A5000    Off  | 00000000:04:00.0 Off |                  Off |
| 42%   71C    P2   212W / 230W |  17106MiB / 24256MiB |     80%      Default |
|                               |                      |                  N/A |
+-------------------------------+-------------

In [14]:
epochList = []
foldList = []
corrList = []
for epoch in range(EPOCHS): 
    for fold in range(FOLDS): 
        df = pd.DataFrame(metrics[fold][epoch]).T
        df.columns = ["loss", "pred", "true"]
        #corr = np.corrcoef(df["pred"], df["true"])

        predCols = ["pred" + item for item in groundTruths]
        gtCols = ["gt" + item for item in groundTruths]

        df[predCols] = pd.DataFrame(df["pred"].tolist(), index=df.index)
        df[gtCols] = pd.DataFrame(df["true"].tolist(), index=df.index)

        corrScores = []
        for colName in groundTruths: 
            corr = np.corrcoef(df["pred" + colName], df["gt" + colName])[1, 0]
            corrScores.append(corr)
        epochList.append(epoch)
        foldList.append(fold)
        corrList.append(corrScores)

In [15]:
df = pd.DataFrame({"epoch":epochList, "fold":foldList, "corrList":corrList})
df[gtCols] = pd.DataFrame(df["corrList"].tolist(), index=df.index)
df = df.drop(columns=["corrList"])

In [16]:
df.groupby(by="epoch").agg(np.mean)

,fold,gtGeography,gtEntities,gtTime,gtNarrative,gtOverall
epoch,,,,,,
0,2,0.019156,0.028727,0.044655,0.027490,0.050349
1,2,0.682826,0.831071,0.344686,0.822095,0.865789
2,2,0.753115,0.852055,0.371539,0.843999,0.880246
